# **Notebook de test de Téléchargement de donnée de basket depuis le site nba.com**

Sommaire

## Imports

In [42]:
%load_ext autoreload
%autoreload 2

import sys, os, re, time #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
import Connexion_Transfert as ct
from TelechargementDonnees import (JourneeSiteNba, PasDeMatchError, JoueursSiteNba, 
                                   DriverFirefox, gererCookie, simplifierNomJoueur, Blessures)
import pandas as pd
import numpy as np
import time
from collections import Counter
from datetime import date

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException, ElementClickInterceptedException
from selenium.webdriver.support.ui import Select

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Telecharger les données liees aux matchs depuis le site internet pour une ou plusieurs dates  
1. on telecharge une journee
1. on sauvegarder dans un json

In [ ]:
#telecharger une journee
try : 
    journee=JourneeSiteNba('2021-02-10', dossierExportCsv=r'C:\Users\martin.schoreisz\git\Basket\Basket\data\testUsa')
    #exporter
    #journee.saveCsv()
except PasDeMatchError as e : 
    print(e)

In [ ]:
# pour un ensemble de journee : 
for j in ['2021-01-%02d' %n  for n in range(25,27)] : 
    print(j)
    try :
        journee=JourneeSiteNba(j,dossierExportCsv=r'C:\Users\martin.schoreisz\git\Basket\Basket\data\testUsa')
        journee.saveCsv()
    except PasDeMatchError as e: 
        print(e)    
        continue

## 2. Telecharger les données liees aux joueurs depuis le site internet 
> Dans le cas ou on souhaite telecharger la totalite des donnees de la page des joueuers
1. on se connecte à la page principale des joueurs
1. on affiche l'integralite des donnees

In [62]:
#creation de la classe
pagejoueuer=EnsembleJoueursSiteNba()
#acces aux donnees
pagejoueuer.dfJoueurs
#export des donnees
pagejoueuer.dfJoueurs.reset_index(drop=True).to_json(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\temp\basket\Joueurs_saison_2020-2021\Joueurs_saison_2020-2021.json')

## 3. Test donnees blessure sur site americain
1. on se connecte à la page des blessures par equipes pour recuperer tous les jouuers blesses et mettre a jour la table existante 
1. on se connecte à l apage des joueurs blesses par jour on prend que le plus recent.

Dans les deux cas on cherche les joueurs non presents dans la base des blesses et on fait des insert, ou on chreche les joueurs presents dans la base avec une date de guerison inconnues et on met a jour le type de blessures.

In [7]:
dfsInjuries=Blessures().dfInjuries

In [37]:
#récupérer les blesses en cours existants dans la base puis mettre a jour
with ct.ConnexionBdd('basket') as c :
    rqtBlesseEnCoursBdd="""SELECT b.id_blessure, j.nom, j.nom_simple
                             FROM donnees_source.blessure b JOIN donnees_source.joueur j ON b.id_joueur=j.id_joueur
                             WHERE date_guerison IS NULL and id_type_blessure=99"""
    blesseEnCoursBdd=pd.read_sql(rqtBlesseEnCoursBdd, c.sqlAlchemyConn)
    typeBlessure=pd.read_sql("""SELECT id_type_blessure, nom_blessure, LOWER(nom_blessure_anglais) "Injury"
                                FROM donnees_source.enum_type_blessure""", c.sqlAlchemyConn)
    #mettre à jour les joueurs connus : 
    #creer la df finale de MaJ
    tempDfBlessePourMaJ=blesseEnCoursBdd.merge(dfsInjuries, on='nom_simple')

In [37]:
    #insrere cette df dans la base
    if not tempDfBlessePourMaJ.empty : 
        tempDfBlessePourMaJ.to_sql('blesses_temp', c.sqlAlchemyConn, schema='public', if_exists='replace', index=False)
        #update la table
        rqtUpdate="""UPDATE donnees_source.blessure b
                      SET id_type_blessure=t.id_type_blessure
                      FROM public.blesses_temp t
                      WHERE b.id_blessure=t.id_blessure"""
        c.sqlAlchemyConn.execute(rqtUpdate)
        #drop la table temporaire
        c.sqlAlchemyConn.execute("DROP TABLE IF EXISTS public.blesses_temp")

In [9]:
#recuperer les blesses du site non presents dans la Bdd, inserer ceux pour qui ont a une date du dernier match joue
with ct.ConnexionBdd('basket') as c :
    rqtBlesseEnCoursBdd="""SELECT b.id_blessure, j.nom, j.nom_simple
                             FROM donnees_source.blessure b JOIN donnees_source.joueur j ON b.id_joueur=j.id_joueur
                             WHERE date_guerison IS NULL and id_type_blessure!=99"""
    blesseEnCoursBdd=pd.read_sql(rqtBlesseEnCoursBdd, c.sqlAlchemyConn)
    dfInjuriesInconnu=dfsInjuries.loc[~dfsInjuries.nom_simple.isin(blesseEnCoursBdd.nom_simple.tolist())]
   

In [45]:
dfInjuriesInconnu.to_sql('blesses_inconnus_temp', c.sqlAlchemyConn, schema='public', if_exists='replace', index=False)
    rqtBlessesInconnusAInserer="""WITH 
                                tous_matchs_blesses_inconnus as(
                                SELECT *
                                 FROM public.blesses_inconnus_temp bi JOIN (SELECT s.*, m.date_match
                                      FROM donnees_source.stat_nom s JOIN donnees_source."match" m ON s.id_match=m.id_match) s 
                                      ON s.nom_simple=bi.nom_simple),
                                blesses_inconnus_dernier_match AS (
                                SELECT DISTINCT ON (id_joueur) id_joueur, "Injury", date_match, date_match+1 date_blessure
                                 FROM tous_matchs_blesses_inconnus
                                 ORDER BY id_joueur, date_match DESC),
                                blesses_inconnus_insertion as(
                                SELECT b.id_joueur,b.date_blessure,e.id_type_blessure
                                 FROM blesses_inconnus_dernier_match b JOIN donnees_source.enum_type_blessure e ON b."Injury"=lower(e.nom_blessure_anglais))
                                INSERT INTO donnees_source.blessure (id_joueur, date_blessure, id_type_blessure)
                                 SELECT id_joueur, date_blessure, id_type_blessure FROM blesses_inconnus_insertion"""
    c.sqlAlchemyConn.execute(rqtBlessesInconnusAInserer)

In [55]:
#reste ceux pour lesquels on a pas de date : on met le jour de updated
with ct.ConnexionBdd('basket') as c :
    rqtBlesseEnCoursBdd="""SELECT b.id_blessure, j.nom, j.nom_simple
                             FROM donnees_source.blessure b JOIN donnees_source.joueur j ON b.id_joueur=j.id_joueur
                             WHERE date_guerison IS NULL and id_type_blessure!=99"""
    blesseEnCoursBdd=pd.read_sql(rqtBlesseEnCoursBdd, c.sqlAlchemyConn)
    joueurs=pd.read_sql('select nom_simple, id_joueur from donnees_source.joueur', c.sqlAlchemyConn)
    dfInjuriesInconnu=dfsInjuries.loc[~dfsInjuries.nom_simple.isin(blesseEnCoursBdd.nom_simple.tolist())].merge(
                        joueurs, on='nom_simple')

In [55]:
    dfInjuriesInconnu['date_blessure']=dfInjuriesInconnu.Updated.apply(lambda x : pd.to_datetime(' '.join(x.split(', ')[1].split()
                                                                                    [::-1]+[str(date.today().year),])))
    dfInjuriesInconnu[['id_joueur','date_blessure','id_type_blessure']].to_sql('blessure', c.sqlAlchemyConn, 
                                                                    schema='donnees_source', if_exists='append', index=False)